In [ ]:
# Cell 2: Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch.nn.functional as F
import os


In [ ]:
# Cell 3: Define Data Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust size as needed
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Standard ImageNet normalization
])


In [ ]:
# Cell 4: Load Train, Validation, and Test Datasets
train_dir = '/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Train'
val_dir = '/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Validation'
test_dir = '/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Test'

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)


In [ ]:
# Cell 5: Define Basic Block for ResNet-like Model
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        return F.relu(out)

In [ ]:
# Cell 6: Define ResNeXt Block with Grouped Convolutions
class ResNeXtBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, groups=32, downsample=None):
        super(ResNeXtBlock, self).__init__()
        width = out_channels // 2
        self.conv1 = nn.Conv2d(in_channels, width, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2 = nn.Conv2d(width, width, kernel_size=3, stride=stride, padding=1, groups=groups, bias=False)
        self.bn2 = nn.BatchNorm2d(width)
        self.conv3 = nn.Conv2d(width, out_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        return F.relu(out)

In [ ]:
# Cell 7: Define Custom ResNet-18 Model
class CustomResNet18(nn.Module):
    def __init__(self, num_classes=2):
        super(CustomResNet18, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(BasicBlock, 64, 64, 2)
        self.layer2 = self._make_layer(BasicBlock, 64, 128, 2, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 128, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 256, 512, 2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, in_channels, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        layers = [block(in_channels, out_channels, stride, downsample)]
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

In [ ]:
# Cell 8: Define Custom ResNeXt Model
class CustomResNeXt(nn.Module):
    def __init__(self, num_classes=2):
        super(CustomResNeXt, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(ResNeXtBlock, 64, 128, 3, stride=1, groups=32)
        self.layer2 = self._make_layer(ResNeXtBlock, 128, 256, 4, stride=2, groups=32)
        self.layer3 = self._make_layer(ResNeXtBlock, 256, 512, 6, stride=2, groups=32)
        self.layer4 = self._make_layer(ResNeXtBlock, 512, 1024, 3, stride=2, groups=32)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, num_classes)

    def _make_layer(self, block, in_channels, out_channels, blocks, stride, groups):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        layers = [block(in_channels, out_channels, stride, groups, downsample)]
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels, groups=groups))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

In [ ]:
# Cell 9: Define the Ensemble Model
class EnsembleModel(nn.Module):
    def __init__(self, resnet, resnext, num_classes=2):
        super(EnsembleModel, self).__init__()
        self.resnet = resnet
        self.resnext = resnext
        self.fc = nn.Sequential(
            nn.Linear(4, 512),  # Input size changed to 4 to match combined_features shape
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        resnet_features = self.resnet(x)
        resnext_features = self.resnext(x)

        # Print shapes for debugging
        print("ResNet output shape:", resnet_features.shape)
        print("ResNeXt output shape:", resnext_features.shape)

        combined_features = torch.cat((resnet_features, resnext_features), dim=1)
        print("Combined features shape:", combined_features.shape)

        return self.fc(combined_features)

In [ ]:
# Cell 10: Initialize Models and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet = CustomResNet18(num_classes=2).to(device)
resnext = CustomResNeXt(num_classes=2).to(device)
ensemble_model = EnsembleModel(resnet, resnext).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ensemble_model.parameters(), lr=0.001)

In [ ]:
# Cell 11: Define Training Function
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    return running_loss / len(train_loader.dataset)


In [ ]:
# Cell 12: Define Validation Function
def validate_model(model, val_loader, device):
    model.eval()
    all_labels, all_preds = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return accuracy, precision, recall, f1

In [ ]:
# Cell 13: Train and Evaluate the Model
num_epochs = 10  # Set as needed
for epoch in range(num_epochs):
    train_loss = train_model(ensemble_model, train_loader, criterion, optimizer, device)
    accuracy, precision, recall, f1 = validate_model(ensemble_model, val_loader, device)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features sha

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features sha

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


...

ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features sha

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features sha

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features shape: torch.Size([32, 4])
ResNet output shape: torch.Size([32, 2])
ResNeXt output shape: torch.Size([32, 2])
Combined features sha

In [ ]:
# Cell 10: Test the Model
test_accuracy, test_precision, test_recall, test_f1 = validate_model(ensemble_model, test_loader, device)
print(f'Test Accuracy: {test_accuracy:.4f}, Test Precision: {test_precision:.4f}, Test Recall: {test_recall:.4f}, Test F1 Score: {test_f1:.4f}')


In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# Function to test a single image
def test_image(model, image_path, device):
    model.eval()  # Set the model to evaluation mode
    # Define the same transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize as per your model's input
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
    ])

    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')  # Ensure the image is in RGB mode
    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)  # Move the image to the same device as the model

    with torch.no_grad():  # No need to track gradients
        output = model(image)  # Get the model output
        _, predicted = torch.max(output, 1)  # Get the predicted class

    return predicted.item()  # Return the predicted class as an integer

# Example usage
image_path = '/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Validation/Real/real_200.jpg'
predicted_class = test_image(ensemble_model, image_path, device)
if(predicted_class==0):
  print('Fake')
else:
  print('Real')
print(f'Predicted class: {predicted_class}')


ResNet output shape: torch.Size([1, 2])
ResNeXt output shape: torch.Size([1, 2])
Combined features shape: torch.Size([1, 4])
Real
Predicted class: 1
